## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-03-13-01-11 +0000,bbc,Luigi Mangione's likeness used to model shirt ...,https://www.bbc.com/news/articles/c4g5vr4vzpzo...
1,2025-09-03-13-00-42 +0000,bbc,"Xi, Putin and Kim show united front at huge Ch...",https://www.bbc.com/news/articles/cp8z83np4xjo...
2,2025-09-03-13-00-00 +0000,wsj,Tax-Exempt Munis vs. Taxable Bonds: Which Has ...,https://www.wsj.com/finance/investing/tax-exem...
3,2025-09-03-13-00-00 +0000,missionlocal,Mouth hole,https://missionlocal.org/2025/09/sf-photos-mou...
4,2025-09-03-12-59-58 +0000,nyt,Pro-Palestinian Protesters at the Venice Film ...,https://www.nytimes.com/2025/08/30/arts/gaza-p...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2314/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
68,trump,50
143,china,22
15,military,16
9,kim,15
217,new,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
97,2025-09-03-05-07-00 +0000,nypost,"Trump accuses Putin, Kim Jong Un and Xi Jinpin...",https://nypost.com/2025/09/03/us-news/trump-ac...,166
309,2025-09-02-14-16-14 +0000,latimes,Trump deployment of military troops to Los Ang...,https://www.latimes.com/california/story/2025-...,110
118,2025-09-03-02-57-51 +0000,nypost,China’s Xi hosts largest-ever military parade ...,https://nypost.com/2025/09/02/world-news/china...,108
89,2025-09-03-06-21-08 +0000,bbc,'A clear message to Trump': BBC correspondents...,https://www.bbc.com/news/articles/c7545p2px5no...,106
270,2025-09-02-17-48-32 +0000,bbc,Trump's use of National Guard in Los Angeles w...,https://www.bbc.com/news/articles/c5ylyd9lkkqo...,104


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
97,166,2025-09-03-05-07-00 +0000,nypost,"Trump accuses Putin, Kim Jong Un and Xi Jinpin...",https://nypost.com/2025/09/03/us-news/trump-ac...
270,54,2025-09-02-17-48-32 +0000,bbc,Trump's use of National Guard in Los Angeles w...,https://www.bbc.com/news/articles/c5ylyd9lkkqo...
94,51,2025-09-03-05-23-07 +0000,nypost,Vice President JD Vance and second lady Usha w...,https://nypost.com/2025/09/03/us-news/vice-pre...
162,49,2025-09-02-23-16-19 +0000,nypost,Dramatic footage shows US strike that oblitera...,https://nypost.com/2025/09/02/world-news/foota...
303,46,2025-09-02-14-38-46 +0000,bbc,Ex-actor Zack Polanski's unusual path to becom...,https://www.bbc.com/news/articles/clyrev00lwno...
163,42,2025-09-02-23-13-56 +0000,nypost,Jeffrey Epstein’s victims revealed ‘additional...,https://nypost.com/2025/09/02/us-news/jeffrey-...
86,41,2025-09-03-06-39-35 +0000,nypost,Felon charged in 9-year-old’s train station de...,https://nypost.com/2025/09/03/us-news/chicago-...
123,38,2025-09-03-02-23-51 +0000,nypost,Florida man allegedly runs over foot model wit...,https://nypost.com/2025/09/02/us-news/florida-...
189,33,2025-09-02-21-29-07 +0000,nypost,Would-be assassin Ryan Wesley Routh challenges...,https://nypost.com/2025/09/02/us-news/would-be...
283,31,2025-09-02-16-46-17 +0000,bbc,UK borrowing costs hit 27-year high adding to ...,https://www.bbc.com/news/articles/cy989njnq2wo...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
